In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy import signal
from scipy.fft import fft, fftfreq

## Sensor Labels
    EMG 1 = a; left side of face
    EMG 2 = b; right side of face
    Ch a-d = 1-4; a-d, indicated by sensor color

In [12]:
class Emg():
    FSAMP = 2000
    def __init__(self,path = 'subject1_smile_1.xlsx'):
        self.df = pd.read_excel(path)
        self.TTIME = self.df.time.iloc[-1]
        self.NSAMP = self.df.count().a1
        self.df = self.df.set_index(self.df.time)
        self.df = self.df.drop(['time'],axis=1)
        self.RAW = self.df
        self.df = self.df*(10**6)
    def print_char(self):
        self.CHAR = {
          "T": self.TTIME,
          "N": self.NSAMP,
          "fs": self.FSAMP
        }
        print(self.CHAR)

    def plot_single(self,x,y):
        for col in self.df.columns:
            plt.figure()
            plt.plot(x,y[cols])
            plt.grid()
            plt.title('Smile: '+ '\n'+'{} '.format(col))
            plt.xlabel('time (s)')
            plt.ylabel('Voltage (uV)')
    def create_bandpass(self,low_band = 3, high_band = 400, order_bp = 2):
        """ return numerator, denominator = z,w of the IIR Bessel filter"""
        self.Z, self.W = sp.signal.bessel(order_bp, [low_band,high_band], btype='bandpass',analog = False, output= 'ba', fs = self.FSAMP)
        self.LOWBAND = low_band
        self.HIGHBAND = high_band
        return self.Z, self.Z
    def apply_bandpass(self):
        self.BANDPASSED = pd.DataFrame(columns = self.df.columns, index = self.df.index)
        for col in self.df.columns:
            self.BANDPASSED[col] = sp.signal.filtfilt(self.Z,self.W,self.df[col],padlen=0)
        return self.BANDPASSED
    def rectify(self):
        self.RECT = pd.DataFrame(columns = self.df.columns, index = self.df.index)
        self.RECT = np.abs(self.BANDPASSED)
        return self.RECT    
    def linear_envelope(self, window_size = 100):
        window = np.ones(window_size)/float(window_size)
        self.ENVELOPE = pd.DataFrame(columns = self.df.columns, index = self.df.index)
        for col in self.RECT.columns:
            self.ENVELOPE[col] = np.sqrt(np.convolve(self.RECT[col]**2,window,'same'))
        self.OUTPUT = self.ENVELOPE
        return self.ENVELOPE
            
    def process(self):
        emg.create_bandpass()
        emg.apply_bandpass()
        emg.rectify()
        return self.OUTPUT
    
class main(Emg):
    def plots(df):
        for col in df.columns:
            df[col].plot()
            """plt.figure()
            plt.plot(df.index,df[cols])
            plt.grid()
            plt.title('Smile: '+ '\n'+'{} '.format(col))
            plt.xlabel('time (s)')
            plt.ylabel('Voltage (uV)')"""
    def runall(self):
        emg = Emg()
        emg.create_bandpass()
        emg.apply_bandpass()
        emg.rectify()
        self.OUTPUT = emg.linear_envelope()
        return self.OUTPUT

In [13]:
m = main()
out = m.runall()

In [16]:
out

,a1,a2,a3,a4,b1,b2,b3,b4
time,,,,,,,,
0.0000,3.875326,5.529149,4.951291,4.686864,3.075379,10.077514,2.449169,3.516867
0.0005,3.915677,5.580736,4.976485,4.698259,3.084133,10.290070,2.456378,3.552205
0.0010,3.949768,5.649743,5.002898,4.717218,3.095258,10.523841,2.461028,3.595939
0.0015,3.976041,5.730427,5.030437,4.748348,3.114958,10.736159,2.466006,3.655218
0.0020,3.995975,5.807806,5.060150,4.790211,3.153382,10.887676,2.470424,3.726788
...,...,...,...,...,...,...,...,...
27.4945,0.104172,0.141215,0.158062,0.136885,0.086923,0.074001,0.079691,0.042851
27.4950,0.102481,0.138960,0.155410,0.134670,0.085484,0.072982,0.078371,0.042140
27.4955,0.100752,0.136651,0.152704,0.132404,0.084014,0.071927,0.077022,0.041415
